In [1]:
%%capture
pip install ddgs


In [2]:
# --- CELL 1: SETUP & AUTHENTICATION (CORRECT ORDER) ---
import os
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

# 1. Retrieve Secrets & Set Env Vars FIRST
# (We do this BEFORE importing the kaggle library to prevent the crash)
user_secrets = UserSecretsClient()

try:
    # Get secrets
    gemini_key = user_secrets.get_secret("GOOGLE_API_KEY")
    k_user = user_secrets.get_secret("KAGGLE_USERNAME")
    k_key = user_secrets.get_secret("KAGGLE_KEY")

    # Set Environment Variables
    os.environ["KAGGLE_USERNAME"] = k_user
    os.environ["KAGGLE_KEY"] = k_key
    print("✅ Environment variables set.")
    
    # 2. NOW it is safe to import the Kaggle API
    from kaggle.api.kaggle_api_extended import KaggleApi
    
    k_api = KaggleApi()
    k_api.authenticate()
    print("✅ Kaggle API Authenticated.")

    # 3. Authenticate Gemini
    genai.configure(api_key=gemini_key)
    print("✅ Gemini API Authenticated.")

except Exception as e:
    print(f"❌ Error: {e}")

✅ Environment variables set.
✅ Kaggle API Authenticated.
✅ Gemini API Authenticated.


In [3]:
# --- 1. IMPORTS & SETUP ---
import os
import json
import glob
import time
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from ddgs import DDGS
from kaggle.api.kaggle_api_extended import KaggleApi
from kaggle_secrets import UserSecretsClient

# --- 2. AUTHENTICATION & CONFIG ---
def setup_system():
    user_secrets = UserSecretsClient()
    try:
        # Gemini Auth
        api_key = user_secrets.get_secret("GOOGLE_API_KEY")
        genai.configure(api_key=api_key)
        
        # Kaggle Auth
        k_user = user_secrets.get_secret("KAGGLE_USERNAME")
        k_key = user_secrets.get_secret("KAGGLE_KEY")
        
        # Set Env Vars (Safest method)
        os.environ["KAGGLE_USERNAME"] = k_user
        os.environ["KAGGLE_KEY"] = k_key
        
        # Initialize API
        api = KaggleApi()
        api.authenticate()
        return api
    except Exception as e:
        print(f"❌ Auth Error: {e}")
        return None

k_api = setup_system()

# UPDATED: Use the stable 1.5 Flash model
model_name = 'gemini-2.5-flash' 

# Safety Config (Block None to allow medical/technical discussions)
safety_config = {
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
}

# --- HELPER: SAFE GENERATION ---
# --- HELPER: ROBUST STREAMING GENERATION ---
def safe_generate(model, prompt):
    """
    Uses streaming to bypass 503 Upstream Timeouts on long code generation.
    Does NOT rely on 'timeout' parameter, but on active connection keep-alive.
    """
    try:
        # Enable streaming. This creates a generator that yields text chunks.
        response_iterator = model.generate_content(prompt, safety_settings=safety_config, stream=True)
        
        full_text = ""
        # Iterate through the chunks as they arrive.
        # This loop keeps the HTTP connection active, preventing timeouts.
        for chunk in response_iterator:
            if chunk.text:
                full_text += chunk.text
        
        if not full_text:
            return "Error: Model returned empty response."
            
        return full_text

    except Exception as e:
        return f"Error: Generation failed with {str(e)}"

# --- 3. AGENT DEFINITIONS ---

class ProblemScoutAgent:
    def run(self, slug):
        print(f"🕵️ [Scout] Analyzing competition: {slug}...")
        try:
            # RSNA Override
            if "rsna" in slug and "aneurysm" in slug:
                return "Goal: Detect intracranial aneurysms on 3D CT. Metric: Weighted Log Loss. Data: 85GB 3D DICOM."
            
            # API Fetch
            comps = k_api.competitions_list(search=slug.split('-')[0])
            target = next((c for c in comps if c.ref == slug), None)
            if target:
                return f"Goal: {target.description[:500]}... Metric: {target.evaluationMetric}."
            return "Could not fetch official details via API."
        except Exception as e:
            return f"Scout Error: {e}"

class ForumScoutAgent:
    def run(self, slug):
        print(f"📡 [Forum] Scanning discussions for: {slug}...")
        readable_name = slug.replace("-", " ")
        query = f"{readable_name} kaggle discussion solution tricks"
        try:
            with DDGS() as ddgs:
                results = list(ddgs.text(query, max_results=5))
            
            if not results: return "No discussions found."
            summary = "\n".join([f"- {r['title']}: {r['body'][:200]}..." for r in results])
            return summary
        except Exception as e:
            return f"Forum Search Error: {e}"

class NotebookScoutAgent:
    def run(self, slug):
        print(f"👨‍💻 [Notebook] Hunting for top code...")
        try:
            kernels = []
            try: kernels = k_api.kernels_list(competition=slug, sort_by='voteCount', page_size=1)
            except: pass
            
            if not kernels:
                print("   (Strict filter failed, trying broad search...)")
                kernels = k_api.kernels_list(search=slug.split('-')[0], sort_by='voteCount', page_size=1)
            
            if not kernels: return "No public code found."
            
            top_k = kernels[0]
            print(f"   (Found: {top_k.title})")
            
            k_api.kernels_pull(top_k.ref, path="./downloaded_code")
            
            code_content = ""
            files = glob.glob("./downloaded_code/*")
            for f_path in files:
                if f_path.endswith(".py"):
                    with open(f_path,'r') as f: code_content += f.read()
                elif f_path.endswith(".ipynb"):
                    with open(f_path,'r') as f:
                        nb = json.load(f)
                        for c in nb['cells']: 
                            if c['cell_type']=='code': code_content += "".join(c['source']) + "\n"
            
            if len(code_content) < 50: return "Code was empty."
            return f"--- CODE FROM {top_k.title} ---\n{code_content[:25000]}"
        except Exception as e:
            return f"Code Download Error: {e}"

class StrategistAgent:
    def run(self, slug, goal, forum_intel, code_intel):
        print(f"🧠 [Strategist] Formulating plan...")
        
        prompt = f"""
        Act as a Kaggle Grandmaster.
        Competition: {slug}
        
        1. OFFICIAL GOAL: {goal}
        2. COMMUNITY INTEL: {forum_intel}
        3. EXISTING CODE BASELINE: {code_intel[:2000]}... (truncated)
        
        Task: Write a 'Winning Strategy' report.
        - Critique the baseline.
        - Identify the specific model architecture we should build.
        - Suggest 1 specific data augmentation or feature engineering technique.
        - Define the validation strategy.
        """
        
        model = genai.GenerativeModel(model_name)
        return safe_generate(model, prompt)

class CodeGeneratorAgent:
    def run(self, strategy_report):
        print(f"🏗️ [Builder] Writing final solution.py...")
        
        prompt = f"""
        You are an expert Python Developer.
        
        Based on this strategy:
        {strategy_report}
        
        Write a COMPLETE, RUNNABLE 'main.py' script.
        - Include Dataset class, Model class, and Training Loop.
        - Use PyTorch.
        - Handle the specific data types mentioned (e.g. Images, CSVs).
        """
        
        model = genai.GenerativeModel(model_name)
        return safe_generate(model, prompt)

# --- 4. THE ORCHESTRATOR ---

class KaggleCommandSystem:
    def __init__(self):
        self.scout = ProblemScoutAgent()
        self.forum = ForumScoutAgent()
        self.notebook = NotebookScoutAgent()
        self.strategist = StrategistAgent()
        self.builder = CodeGeneratorAgent()
        
    def execute(self, competition_slug):
        print(f"🚀 STARTING KAGGLE COMMAND FOR: {competition_slug}\n" + "="*50)
        
        goal_data = self.scout.run(competition_slug)
        forum_data = self.forum.run(competition_slug)
        code_data = self.notebook.run(competition_slug)
        
        strategy = self.strategist.run(competition_slug, goal_data, forum_data, code_data)
        
        # If strategy failed, don't try to build code
        if "Error" in strategy and len(strategy) < 100:
            return {"strategy_report": strategy, "final_code": "Skipped due to strategy error."}
            
        final_code = self.builder.run(strategy)
        
        return {
            "strategy_report": strategy,
            "final_code": final_code
        }

# --- 5. EXECUTION ---
system = KaggleCommandSystem()
slug = "rsna-intracranial-aneurysm-detection" 
# You can change 'slug' to 'hull-tactical-speed-dating' or any other active comp to test

try:
    result = system.execute(slug)
    
    from IPython.display import Markdown, display
    print("\n" + "="*50)
    display(Markdown(f"## 📄 STRATEGY REPORT\n{result['strategy_report']}"))
    print("\n" + "="*50)
    display(Markdown(f"## 🐍 GENERATED CODE\n{result['final_code']}"))
    
    with open("submission.py", "w") as f:
        f.write(result['final_code'])
    print("✅ Code saved to 'submission.py'")

except Exception as e:
    print(f"❌ Fatal System Error: {e}")

🚀 STARTING KAGGLE COMMAND FOR: rsna-intracranial-aneurysm-detection
🕵️ [Scout] Analyzing competition: rsna-intracranial-aneurysm-detection...
📡 [Forum] Scanning discussions for: rsna-intracranial-aneurysm-detection...
👨‍💻 [Notebook] Hunting for top code...
   (Found: RSNA Aneurysm Detection Demo Submission)
🧠 [Strategist] Formulating plan...
🏗️ [Builder] Writing final solution.py...



## 📄 STRATEGY REPORT
As a Kaggle Grandmaster, let's dissect the RSNA Intracranial Aneurysm Detection challenge and formulate a winning strategy. This competition demands not just accuracy but also robust, efficient inference on 3D CT data.

---

## Winning Strategy: RSNA Intracranial Aneurysm Detection

**Competition Overview:**
The goal is to detect and classify intracranial aneurysms across various anatomical locations on 3D CT scans, ultimately predicting the presence and location of aneurysms with a Weighted Log Loss metric. The data consists of substantial 85GB 3D DICOM files, and inference is online with a 30-minute time limit per series.

### 1. Critique of the Existing Code Baseline

The provided baseline is **minimalistic and purely serves as an API scaffold**, not a predictive solution.
*   **No Predictive Logic:** It contains no actual model loading, DICOM processing, or inference code. It merely demonstrates the function signature (`predict(series_path) -> pl.DataFrame | pd.DataFrame`) and the expected output format (a DataFrame with `LABEL_COLS`).
*   **Online Inference Constraint Highlighted:** The explicit mention of the "30 minutes per series" constraint is the most valuable piece of information from the baseline. This immediately signals that model efficiency and streamlined data processing will be paramount for a successful submission. Ensembles of very large models might be challenging to deploy effectively under this time limit.
*   **DICOM Handling Implied:** The `DICOM_TAG_ALLOWLIST` indicates that core image data and relevant metadata will be present, but potentially stripped of sensitive or less common tags. The baseline, however, doesn't demonstrate how to load and convert the series of 2D DICOM slices into a coherent 3D volume.

**In essence, the baseline is a blank canvas. Our strategy must build a complete solution from the ground up, keeping the online inference constraints firmly in mind.**

### 2. Specific Model Architecture

Given the 3D nature of the data and the multi-label classification task, a powerful 3D Convolutional Neural Network (CNN) is essential.

**Recommended Architecture: A 3D ResNeXt/EfficientNet-like Backbone with Multi-Head Classification**

1.  **3D Backbone:** We will leverage a robust 3D CNN architecture for feature extraction.
    *   **Choice:** A 3D adaptation of architectures like **ResNeXt** or **EfficientNet** provides strong capabilities in medical imaging. ResNeXt, with its "cardinality" concept, offers a good balance between accuracy and computational efficiency by aggregating parallel branches. An EfficientNet-like approach scales the model width, depth, and resolution in a principled way.
    *   **Implementation:** The backbone will take the preprocessed 3D CT volume as input. Each convolutional layer will be a 3D convolution (e.g., `Conv3D` in TensorFlow/PyTorch).
    *   **Reasoning:** These architectures are proven to extract hierarchical, context-rich features from images, which is crucial for detecting subtle aneurysms amidst complex brain anatomy. Using 3D convolutions allows the model to learn spatial relationships across slices, essential for aneurysm detection.

2.  **Global Pooling:** After the final convolutional block of the backbone, apply a Global Average Pooling (GAP) layer.
    *   **Reasoning:** GAP effectively summarizes the learned features across the entire 3D volume into a fixed-size vector, making the model robust to variations in input dimensions and reducing the number of parameters in the subsequent classification layers.

3.  **Multi-Head Classification:** Attach multiple independent fully connected (Dense) layers, each with a sigmoid activation, directly after the GAP layer.
    *   **Structure:** Each head will correspond to one of the `LABEL_COLS` (e.g., one head for 'Left Infraclinoid Internal Carotid Artery', another for 'Right Middle Cerebral Artery', etc., and crucially, one for 'Aneurysm Present').
    *   **Reasoning:** This multi-head approach allows the model to learn common underlying features for aneurysm detection, while each head specializes in predicting the presence in a specific anatomical location. The 'Aneurysm Present' head acts as an overall binary classifier, which is a strong signal for the Weighted Log Loss.

4.  **Loss Function:** Weighted Binary Cross-Entropy (BCE) loss.
    *   **Reasoning:** Given the class imbalance (aneurysms are rare, and some locations are rarer than others), applying inverse class frequency weighting to the BCE loss for each head will ensure the model pays adequate attention to the minority positive class.

**Considerations for online inference:**
*   Start with a moderately sized model (e.g., 3D ResNeXt-50 equivalent). Monitor inference time during development.
*   Pre-load the model into memory once at the start of the `kaggle_evaluation.rsna_inference_server` process to avoid repeated loading overhead.

### 3. Specific Data Augmentation or Feature Engineering Technique

**Technique: Multi-Window CT Input Channels**

**Description:** Instead of feeding a single windowed CT volume (e.g., brain window) into the model, we will preprocess the raw Hounsfield Unit (HU) values into *multiple, distinct CT windows* and provide these as separate input channels to the 3D CNN.

**Example Windows:**
1.  **Brain Parenchyma Window:** Typically centered around 40 HU with a width of 80 HU (e.g., levels between 0-80 HU). This highlights grey and white matter.
2.  **Bone Window:** Centered around 400-800 HU with a width of 1500-2000 HU (e.g., levels between -500 to 1500 HU). Useful for identifying skull base structures and potential calcifications related to aneurysms.
3.  **Angiographic/Vessel Window:** A narrower window specifically tuned to highlight vascular structures, often in the range of 80-150 HU.
4.  **Raw Normalized HU:** The original, normalized HU values (e.g., scaled to 0-1 or -1 to 1) for a broader context.

**How it works:**
Each 3D CT scan will be transformed into a 3D tensor with, for example, 3 or 4 channels instead of 1. The model's initial 3D convolutional layer will then operate on these multi-channel inputs.

**Benefits:**
*   **Mimics Radiologist Practice:** Radiologists routinely adjust window levels to optimally visualize different tissues and pathologies. This technique allows the model to simultaneously "see" the image through multiple radiological perspectives.
*   **Rich Feature Representation:** Different windows emphasize different anatomical details. Combining them provides the CNN with a much richer and more robust representation of the local environment around potential aneurysms (e.g., vessel wall, surrounding brain tissue, bony landmarks).
*   **Improved Small Object Detection:** Aneurysms can be small. Providing focused contrast for vessels and surrounding structures helps the model pick up subtle cues that might be lost in a single general window.
*   **No Additional Model Complexity:** This is a preprocessing step that simply changes the input tensor shape (e.g., `(D, H, W, 1)` to `(D, H, W, 4)`), requiring no fundamental change to the CNN architecture itself beyond handling more input channels in the first layer.

**Augmentation (briefly):**
In addition to this feature engineering, standard 3D augmentations like random rotations, flips (left-right), slight scaling, intensity shifts, and elastic deformations should be applied during training to improve generalization.

### 4. Validation Strategy

A robust validation strategy is paramount to ensure our model generalizes well to unseen data and correctly estimates its performance on the Weighted Log Loss metric.

**Strategy: Grouped K-Fold Cross-Validation with Stratification**

1.  **Patient-Level Splitting (Grouping):**
    *   **Core Principle:** The most critical rule in medical imaging competitions is to ensure that data from the same patient (or study, if a patient can have multiple unique studies) does not appear in both the training and validation sets. This prevents data leakage and over-optimistic performance estimates.
    *   **Implementation:** Use `GroupKFold` from `sklearn.model_selection`. The `groups` argument should be based on `StudyInstanceUID` or `PatientID` (prioritize `StudyInstanceUID` if available and unique per scan, as one patient might have multiple studies over time).
    *   **Number of Folds:** 5-folds is a good balance between training time and robust validation estimates.

2.  **Stratification:**
    *   **Challenge:** Aneurysm presence, and especially specific anatomical locations, will likely be highly imbalanced classes.
    *   **Implementation:** Stratify the splits based on the 'Aneurysm Present' label. If possible, a multi-label stratification strategy (considering all `LABEL_COLS` or at least the 'Aneurysm Present' and a few most common specific locations) would be ideal to ensure each fold has a representative distribution of positive and negative cases across various aneurysm types. This might require custom stratification logic if `GroupKFold` alone isn't sufficient for multi-label stratification.
    *   **Reasoning:** Stratification ensures that each fold's validation set is representative of the overall label distribution, leading to more reliable performance metrics.

3.  **Metric Calculation:**
    *   **During Validation:** The primary metric for validation will be the **Weighted Log Loss**, as defined by the competition. Implement the exact calculation logic to ensure consistency.
    *   **Secondary Metrics:** Monitor other metrics like AUC (Area Under the Receiver Operating Characteristic curve) or Average Precision (AP) for individual labels, especially for the rare classes, as these can provide insights not fully captured by Log Loss.

4.  **Training Procedure:**
    *   Train a separate instance of the model (with the architecture described above) on each fold of the K-Fold split.
    *   Save the best performing model checkpoint for each fold (based on validation Weighted Log Loss).

5.  **Inference (for Submission):**
    *   **Ensembling:** The most common Grandmaster strategy is to ensemble the predictions from the models trained on each fold. Average the predicted probabilities from each of the N models (e.g., 5-fold models). This reduces variance and typically improves robustness.
    *   **Test-Time Augmentation (TTA):** For each test series, perform inference on multiple augmented versions (e.g., original, flipped left-right, rotated slightly) and average their predictions. This further boosts robustness but adds to inference time. Carefully balance the number of TTA transformations with the 30-minute time limit.

**Example Validation Loop:**

```python
from sklearn.model_selection import GroupKFold
import numpy as np
import pandas as pd # or polars

# Assuming 'train_df' contains 'StudyInstanceUID' and all LABEL_COLS
groups = train_df['StudyInstanceUID'].values
labels = train_df[LABEL_COLS].values # For stratification if possible

kf = GroupKFold(n_splits=5)

for fold, (train_idx, val_idx) in enumerate(kf.split(train_df, groups=groups)):
    print(f"--- Fold {fold+1}/{kf.n_splits} ---")
    
    fold_train_data = train_df.iloc[train_idx]
    fold_val_data = train_df.iloc[val_idx]
    
    # Check for patient leakage:
    assert len(set(fold_train_data['StudyInstanceUID']) & set(fold_val_data['StudyInstanceUID'])) == 0
    
    # 1. Preprocess and load 3D volumes for fold_train_data and fold_val_data
    # 2. Train model on fold_train_data, evaluating on fold_val_data
    #    - Apply Multi-Window CT Input Channels preprocessing
    #    - Apply 3D augmentations during training
    #    - Use Weighted Binary Cross-Entropy loss
    #    - Monitor Weighted Log Loss on validation set
    # 3. Save best model for this fold
    # 4. Store validation predictions and true labels for overall evaluation
```

This strategy focuses on leveraging advanced 3D CNNs, enriching the input data with expert knowledge (multi-windowing), and validating performance rigorously through patient-level grouped cross-validation, while always keeping the tight online inference constraints in mind.

## 🐍 GENERATED CODE
Error: Generation failed with 400 Bad Request

✅ Code saved to 'submission.py'
